<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150 align="right">
<img src=images/Acme.jpeg width=100 align="left">

#  Tokens

 * Almost all operations in Vault requires a token.  It is important to understand the token lifecycle and various parameters.
 

### Vault Setup

### Prerequisites
* Running Vault Enterprise instance (also in dev mode) 


In [5]:
vault -version

Vault v1.6.0 (7ce0bd9691998e0443bc77e98b1e2a4ab1e965d4)


In [4]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_TOKEN=root
export VAULT_SKIP_VERIFY=true

In [6]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.6.0
Storage Type    inmem_transactional_ha
Cluster Name    vault-cluster-1dbc1310
Cluster ID      747ea9f7-7a07-0b5c-dbc6-d14ce8f1f2fe
HA Enabled      true
HA Cluster      https://127.0.0.1:8201
HA Mode         active


In [11]:
vault login root

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       53Vy1NWM65Xd54dUnw9duptm
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


---
### Create Short-Lived Tokens

In [7]:
vault token create -h

Usage: vault token create [options]

  Creates a new token that can be used for authentication. This token will be
  created as a child of the currently authenticated token. The generated token
  will inherit all policies and permissions of the currently authenticated
  token unless you explicitly define a subset list policies to assign to the
  token.

  A ttl can also be associated with the token. If a ttl is not associated
  with the token, then it cannot be renewed. If a ttl is associated with
  the token, it will expire after that amount of time unless it is renewed.

  Metadata associated with the token (specified with "-metadata") is written
  to the audit log when the token is used.

  If a role is specified, the role may override parameters specified here.

HTTP Options:

  -address=<string>
      Address of the Vault server. The default is https://127.0.0.1:8200. This
      can also be specified via the VAULT_ADDR environment variable.

  -agent-address=<string>
      Address

In [8]:
# in seconds
vault token create -ttl=120

Key                  Value
---                  -----
token                s.dalUfuCp9FwO2b2wXgSEoprM
token_accessor       9TDS8GSx7vkyM0g480xEbHEp
token_duration       2m
token_renewable      true
token_policies       ["root"]
identity_policies    []
policies             ["root"]



##### We'll use the value of token above in subsequent commands

In [9]:
#vault token lookup <token_task_1>
vault token lookup s.dalUfuCp9FwO2b2wXgSEoprM


Key                 Value
---                 -----
accessor            9TDS8GSx7vkyM0g480xEbHEp
creation_time       1608644631
creation_ttl        2m
display_name        token
entity_id           n/a
expire_time         2020-12-22T21:45:51.52629+08:00
explicit_max_ttl    0s
id                  s.dalUfuCp9FwO2b2wXgSEoprM
issue_time          2020-12-22T21:43:51.526294+08:00
meta                <nil>
num_uses            0
orphan              false
path                auth/token/create
policies            [root]
renewable           true
ttl                 1m28s
type                service



### Token Renewal

In [85]:
#vault token renew -increment=240 <token_task_2>
vault token renew -increment=100 s.KupeTj2SpKUKWzLTYdgpNbWa


Key                  Value
---                  -----
token                s.KupeTj2SpKUKWzLTYdgpNbWa
token_accessor       3UJ1v3RQ8peMhGOtttyFVWzf
token_duration       4m
token_renewable      true
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [103]:
vault token lookup  s.1eBah74nejO3GSiJFhqyEXtJ

Error looking up token: Error making API request.

URL: POST http://127.0.0.1:8200/v1/auth/token/lookup
Code: 403. Errors:

* bad token


---
### Create a Token with Use Limit


In [104]:
vault token create -use-limit=2

Key                  Value
---                  -----
token                s.h03vBSAtiWFCwRQrFalQbjEY
token_accessor       57qPfNTp6C8ErU3VnR37wcKT
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]



##### Look up information about the token consume 1 of the token's uses.

In [105]:
VAULT_TOKEN=s.h03vBSAtiWFCwRQrFalQbjEY vault token lookup

Key                 Value
---                 -----
accessor            57qPfNTp6C8ErU3VnR37wcKT
creation_time       1608541238
creation_ttl        0s
display_name        token
entity_id           n/a
expire_time         <nil>
explicit_max_ttl    0s
id                  s.h03vBSAtiWFCwRQrFalQbjEY
issue_time          2020-12-21T17:00:38.572645+08:00
meta                <nil>
num_uses            1
orphan              false
path                auth/token/create
policies            [root]
renewable           false
ttl                 0s
type                service



##### Write a key/value path cubby/hole/test to consume another of the token's uses.


In [106]:
VAULT_TOKEN=s.h03vBSAtiWFCwRQrFalQbjEY  vault write cubbyhole/test name="student01"

Success! Data written to: cubbyhole/test



##### Fail to read the key/value at path cubby/hole/test as the token is out of uses.

In [107]:
VAULT_TOKEN=s.h03vBSAtiWFCwRQrFalQbjEY vault read cubbyhole/test

Error reading cubbyhole/test: Error making API request.

URL: GET http://127.0.0.1:8200/v1/cubbyhole/test
Code: 403. Errors:

* permission denied


---
### Create an Orphan Token


In [108]:
vault token create -ttl=90

Key                  Value
---                  -----
token                s.XI0Q0cKxEHcv5oD41i2iLr4k
token_accessor       hEc45UIxDjbyh6ZQfX5g8rVC
token_duration       1m30s
token_renewable      true
token_policies       ["root"]
identity_policies    []
policies             ["root"]



##### Create a child token of the above token with a longer TTL of 180 seconds

In [110]:
VAULT_TOKEN=s.XI0Q0cKxEHcv5oD41i2iLr4k vault token create -ttl=180

Key                  Value
---                  -----
token                s.9FQkBGjkatVGgQ73W4aMX2jq
token_accessor       9RHixEKyafVQNhhjxKtJb0fc
token_duration       3m
token_renewable      true
token_policies       ["root"]
identity_policies    []
policies             ["root"]



##### The remaining steps refer to it as *child_token*

After 90 seconds, the token expires!  This automatically revokes its child token.  If you try to look up the child token, you should receive **bad token** error.


In [111]:
#use <child_token>
vault token lookup s.9FQkBGjkatVGgQ73W4aMX2jq

Error looking up token: Error making API request.

URL: POST http://127.0.0.1:8200/v1/auth/token/lookup
Code: 403. Errors:

* bad token


---
### Create a Token Role and Periodic Token

A common use case of periodic token is long-running processese where genration of a new token can interrupt the entire system flow.  The followings illustrate the creation of a role and periodic token for such long-running process.

In [112]:
vault path-help auth/token

## DESCRIPTION



## PATHS

The following paths are supported by this backend. To view help for
any of the paths below, use the help command with any route matching
the path pattern. Note that depending on the policy of your auth token,
you may or may not be able to access certain paths.

    ^accessors/$
        List token accessors, which can then be
        be used to iterate and discover their properties
        or revoke them. Because this can be used to
        cause a denial of service, this endpoint
        requires 'sudo' capability in addition to
        'list'.

    ^create$
        The token create path is used to create new tokens.

    ^create-orphan$
        The token create path is used to create new orphan tokens.

    ^create/(?P<role_name>\w(([\w-.]+)?\w)?)$
        This token create path is used to create new tokens adhering to the given role.

    ^lookup$
        This endpoint will lookup a token and its properties.

    ^lookup-accessor$
        This endpoint wil


##### Create a token role named `monitor`.  This role has default policy and token renewal period of 24 hours.

In [113]:
vault write auth/token/roles/monitor allowed_policies="default" period="24h"

Success! Data written to: auth/token/roles/monitor


In [114]:
vault token create -role="monitor"

Key                  Value
---                  -----
token                s.eKw7hEFL23LHXt289i38jV4o
token_accessor       CXBBpqdLEUB3toXCqLwwpO8r
token_duration       24h
token_renewable      true
token_policies       ["default"]
identity_policies    []
policies             ["default"]


##### This token can be renewed multiple times indefinitely as long as it gets renewed before it expires.

---
### Enable Username and Password Auth Method

In [115]:
vault auth list

Path      Type     Accessor               Description
----      ----     --------               -----------
token/    token    auth_token_7a7fb713    token based credentials


\
&nbsp;
Userpass auth method allows users to login with username and password.

In [116]:
vault auth enable userpass

Success! Enabled userpass auth method at: userpass/



##### Create a user with name `student01`, password `training` with `default` policies

In [117]:
vault write auth/userpass/users/student01 password="training" policies="default"

Success! Data written to: auth/userpass/users/student01


##### Login with the user `student01` and their password `training`

In [118]:
vault login -method=userpass username=student01 password="training"

WARNING! The VAULT_TOKEN environment variable is set! This takes precedence
over the value set by this command. To use the value set by this command,
unset the VAULT_TOKEN environment variable or set it to the token displayed
below.

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                    Value
---                    -----
token                  s.amM7AGL5UvJpMQhtSdy7dZlV
token_accessor         2I6r74p35m8jyQs36xIgdC94
token_duration         768h
token_renewable        true
token_policies         ["default"]
identity_policies      []
policies               ["default"]
token_meta_username    student01


---
### Generate batch tokens

Batch token explanation ....

In [119]:
vault token create -type=batch -policy=default -ttl=360

Key                  Value
---                  -----
token                b.AAAAAQJa73Mz-uEnA6-Cqdqp3ho4oMM4WnXplQFDZzH3vRNAfThUYqg0W3aJz3Hg87Z3c254xMYQXnm34GLSzpDHFG_6hi3G_uUeHNIQWuBVI-rcoFAJ1kexvYDELNKrmg
token_accessor       n/a
token_duration       6m
token_renewable      false
token_policies       ["default"]
identity_policies    []
policies             ["default"]


##### Enable another `userpass` auth method at 'userpass-batch' which generates batch tokens.
<img src=images/think.png width=100>

In [120]:
vault auth enable -path="userpass-batch" -token-type=batch userpass

Success! Enabled userpass auth method at: userpass-batch/


##### Create a user called `john` with the password `training`

In [121]:
vault write auth/userpass-batch/users/john password="training" policies="default"

Success! Data written to: auth/userpass-batch/users/john


In [64]:
vault write transit/keys/orders/config min_decryption_version=2

Success! Data written to: transit/keys/orders/config


&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 class="center">